In [1]:
# Dependencies
import requests
import json
import pandas as pd

# Google API Key
from config import gkey
from pprint import pprint

In [2]:
# read in the csv of winery_final
winery_data = pd.read_csv("../../../Data/csv/Cal_Addr_All_Clean.csv",dtype="object", encoding="utf-8")

In [3]:
winery_data.head()

,PERMIT NUMBER,OWNER NAME,OPERATING NAME,STREET,CITY,STATE,ZIP,COUNTY
0,CA-W-685,"DOMAINE CHANDON, INC.",DOMAINE CHANDON,1 CALIFORNIA DR,YOUNTVILLE,CA,94599,NAPA
1,CA-W-23853,VAN ZYL FAMILY LLC DBA BELONG WINE CO.,BELONG WINE CO.,1 EXECUTIVE WAY,NAPA,CA,94558,NAPA
2,CA-W-23498,PERSON WINE LLC,PERSON WINE COMPANY,1 EXECUTIVE WAY,NAPA,CA,94558,NAPA
3,CA-W-2646,"GEORGE ALTAMURA, SR.",STRACK VINEYARD,1 EXECUTIVE WAY,NAPA,CA,94558,NAPA
4,CA-W-23687,HANABI LAGER COMPANY LLC,LANDLORD SAILOR,1 EXECUTIVE WAY BLDG 1,NAPA,CA,94558,NAPA


In [4]:
# Part 1 set up to look up geocoordinates by adding slots
winery_data['Latitude'] = ""
winery_data['Longitude'] = ""
winery_data.tail(2)

,PERMIT NUMBER,OWNER NAME,OPERATING NAME,STREET,CITY,STATE,ZIP,COUNTY,Latitude,Longitude
4061,CA-W-17697,"MARTIN ARMSTRONG, INC.",TABER RANCH EVENT CENTER,NOT PROVIDED,CAPAY,CA,95607,YOLO,,
4062,CA-W-15575,"WILD HEART WINERY, LLC",WILD HEART WINERY,TWO WEST LOCKEFORD ST,LODI,CA,95240,SAN JOAQUIN,,


In [5]:
for index, row in winery_data.iterrows():
    # set up a parameters dictionary
    
    target_city = row.STREET + " " + row.CITY + ", " + row.STATE + " " + row.ZIP
    # print(target_city)
    params = {"address": target_city, "key": gkey}
    

# base url
    base_url = base_url = "https://maps.googleapis.com/maps/api/geocode/json"

# run a request using our params dictionary
    response = requests.get(base_url, params=params)
    places_data = response.json()
# trap any situations where no info returned
    try:
        winery_data.loc[index,'Latitude'] = places_data['results'][0]['geometry']['location']['lat']
        winery_data.loc[index,'Longitude'] = places_data['results'][0]['geometry']['location']['lng']
     
    except (KeyError, IndexError):
        # if a bad return we do not want this city data, so drop it from the file
        print(f"For winery: {winery_data.loc[index,'OPERATING NAME']}, Missing field/result... deleting.")
        ideal_cities.drop(index, inplace=True)
    # print(f"Target City: {target_city} Latitude: {winery_data.loc[index,'Latitude']} Longitude: {winery_data.loc[index,'Longitude']}")
    

In [6]:
winery_data.head(100)

,PERMIT NUMBER,OWNER NAME,OPERATING NAME,STREET,CITY,STATE,ZIP,COUNTY,Latitude,Longitude
0,CA-W-685,"DOMAINE CHANDON, INC.",DOMAINE CHANDON,1 CALIFORNIA DR,YOUNTVILLE,CA,94599,NAPA,38.3974,-122.365
1,CA-W-23853,VAN ZYL FAMILY LLC DBA BELONG WINE CO.,BELONG WINE CO.,1 EXECUTIVE WAY,NAPA,CA,94558,NAPA,38.2239,-122.257
2,CA-W-23498,PERSON WINE LLC,PERSON WINE COMPANY,1 EXECUTIVE WAY,NAPA,CA,94558,NAPA,38.2239,-122.257
3,CA-W-2646,"GEORGE ALTAMURA, SR.",STRACK VINEYARD,1 EXECUTIVE WAY,NAPA,CA,94558,NAPA,38.2239,-122.257
4,CA-W-23687,HANABI LAGER COMPANY LLC,LANDLORD SAILOR,1 EXECUTIVE WAY BLDG 1,NAPA,CA,94558,NAPA,38.2239,-122.257
...,...,...,...,...,...,...,...,...,...,...
95,CA-W-17645,"ARPENT, LLC",BIRICHINO,1010 INDUSTRIAL WAY,KING CITY,CA,93930,MONTEREY,36.2272,-121.112
96,CA-W-16892,"FILINI WINE COMPANY, LLC",FILINI WINE GROWERS,1010 INDUSTRIAL WAY,KING CITY,CA,93930,MONTEREY,36.2272,-121.112
97,CA-W-3222,"MONTEREY WINE COMPANY, LLC",MMONTEREY WINE COMPANY,1010 INDUSTRIAL WAY,KING CITY,CA,93930,MONTEREY,36.2272,-121.112
98,CA-W-23937,"WARROOM VENTURES, LLC",WARROOM CELLARS,1010 INDUSTRIAL WAY,KING CITY,CA,93930,NaN,36.2272,-121.112


In [7]:
# Set Path to the output file in the analysis folder
Output_Data_Path_Folder = '../../../Data/csv/' 
DataFrame_File_CSV = 'Cal_Addr_All_Clean_Geo.csv'
Output_CSV_Path = os.path.join(Output_Data_Path_Folder,DataFrame_File_CSV)

# Open the data file with write capabilities
winery_data.to_csv(Output_CSV_Path, index = False)